In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, round

# Detener sesiones previas para evitar conflictos de configuración
try:
    spark.stop()
except:
    pass

# Inicializar sesión de Spark con el conector de Cassandra
# Usamos el paquete oficial de Datastax para asegurar compatibilidad
spark = SparkSession.builder \
    .appName("Pipeline_BigData_Fase3") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .config("spark.cassandra.connection.host", "cassandra_db") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .getOrCreate()

print("✅ Sesión de Spark inicializada con el conector de Cassandra.")

✅ Sesión de Spark inicializada con el conector de Cassandra.


In [2]:
# Fase 3.1: Lectura distribuida desde el Keyspace de Cassandra
df_crudo = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="ventas_crudas", keyspace="proyecto_bigdata") \
    .load()

# Fase 3.3: Limpieza de datos (eliminación de montos inválidos)
df_limpio = df_crudo.filter(col("monto_total") > 0)

# Fase 3.2: Lógica de Agregación
# Consolidamos 100k registros en un resumen diario por categoría
df_resumen = df_limpio.groupBy("fecha_venta", "categoria") \
    .agg(
        round(sum("monto_total"), 2).alias("ventas_totales"),
        count("id_venta").alias("cantidad_transacciones")
    )

print("📊 Resumen analítico generado (Vista previa):")
df_resumen.show(10)
df_resumen.printSchema()

📊 Resumen analítico generado (Vista previa):
+-----------+-----------+--------------+----------------------+
|fecha_venta|  categoria|ventas_totales|cantidad_transacciones|
+-----------+-----------+--------------+----------------------+
| 2026-01-02|  Alimentos|     146229.09|                   306|
| 2026-02-03|  Alimentos|     178272.22|                   348|
| 2026-01-29|      Hogar|     158353.48|                   315|
| 2025-12-10|       Ropa|     170576.94|                   353|
| 2026-02-02|Electrónica|     178215.53|                   364|
| 2025-12-12|Electrónica|     169096.27|                   331|
| 2026-01-19|   Deportes|     182792.73|                   366|
| 2026-01-01|       Ropa|      165066.2|                   345|
| 2026-02-05|  Alimentos|     174772.16|                   324|
| 2026-01-09|      Hogar|     169140.95|                   330|
+-----------+-----------+--------------+----------------------+
only showing top 10 rows

root
 |-- fecha_venta: date (null

In [3]:
# Instalación de cliente ligero para ClickHouse (vía protocolo HTTP)
!pip install clickhouse-connect

In [6]:
import clickhouse_connect

# 1. Preparación de datos desde Spark
print("📦 Preparando registros para la migración al Data Warehouse...")
registros = [list(row) for row in df_resumen.collect()]

# 2. Configuración y Carga en ClickHouse
try:
    # Conexión utilizando el Service Name de la red Docker
    client = clickhouse_connect.get_client(
        host='clickhouse_dw', 
        port=8123, 
        username='default', 
        password=''
    )

    # Inicialización del entorno analítico (DW)
    client.command("CREATE DATABASE IF NOT EXISTS dw_analitico")
    
    # Definición de la tabla optimizada para análisis (Motor MergeTree)
    client.command("""
        CREATE TABLE IF NOT EXISTS dw_analitico.ventas_resumen (
            fecha_venta Date,
            categoria String,
            ventas_totales Float64,
            cantidad_transacciones Int64
        ) ENGINE = MergeTree() 
        ORDER BY (fecha_venta, categoria)
    """)

    # Inserción atómica de los datos procesados
    client.insert(
        'dw_analitico.ventas_resumen', 
        registros, 
        column_names=['fecha_venta', 'categoria', 'ventas_totales', 'cantidad_transacciones']
    )

    print(f"✅ CARGA EXITOSA: {len(registros)} registros sincronizados con dw_analitico.")
    print("🚀 Pipeline de integración finalizado: Cassandra -> Spark -> ClickHouse.")

except Exception as e:
    print(f"❌ Error en la sincronización del Data Warehouse: {e}")

📦 Preparando registros para la migración al Data Warehouse...
✅ CARGA EXITOSA: 300 registros sincronizados con dw_analitico.
🚀 Pipeline de integración finalizado: Cassandra -> Spark -> ClickHouse.


In [9]:
import clickhouse_connect
import pandas as pd
from IPython.display import display, HTML

# Conexión al Data Warehouse
client = clickhouse_connect.get_client(host='clickhouse_dw', port=8123, username='default', password='')

print("📊 EJECUTANDO CONSULTAS ANALÍTICAS EN CLICKHOUSE\n")

# --- CONSULTA 1: Análisis de Desempeño por Categoría (Agregación Global) ---
# Cassandra no puede sumar todos los montos de la tabla sin un ALLOW FILTERING (muy lento)
query_1 = '''
    SELECT 
        categoria, 
        sum(ventas_totales) AS ingresos_totales,
        sum(cantidad_transacciones) AS total_operaciones
    FROM dw_analitico.ventas_resumen
    GROUP BY categoria
    ORDER BY ingresos_totales DESC
'''

# --- CONSULTA 2: Promedio de Ticket de Venta por Categoría ---
# ClickHouse calcula promedios sobre millones de registros al instante
query_2 = '''
    SELECT 
        categoria, 
        avg(ventas_totales / cantidad_transacciones) AS ticket_promedio
    FROM dw_analitico.ventas_resumen
    GROUP BY categoria
    ORDER BY ticket_promedio DESC
'''

try:
    # Ejecutar y mostrar Consulta 1
    res1 = client.query(query_1)
    df1 = pd.DataFrame(res1.result_rows, columns=['Categoría', 'Ingresos Totales', 'Transacciones'])
    display(HTML("<h3 style='color: #1565c0;'>1. Ranking de Ingresos por Categoría</h3>"))
    display(df1.style.format({'Ingresos Totales': '${:,.2f}', 'Transacciones': '{:,}'}))

    # Ejecutar y mostrar Consulta 2
    res2 = client.query(query_2)
    df2 = pd.DataFrame(res2.result_rows, columns=['Categoría', 'Ticket Promedio ($)'])
    display(HTML("<h3 style='color: #2e7d32;'>2. Promedio de Venta por Transacción</h3>"))
    display(df2.style.format({'Ticket Promedio ($)': '${:,.2f}'}))

except Exception as e:
    print(f"❌ Error al ejecutar las consultas: {e}")

📊 EJECUTANDO CONSULTAS ANALÍTICAS EN CLICKHOUSE



,Categoría,Ingresos Totales,Transacciones
0,Electrónica,"$40,436,638.07","80,227"
1,Hogar,"$40,386,488.12","80,284"
2,Alimentos,"$40,283,932.62","80,193"
3,Ropa,"$40,258,995.59","80,100"
4,Deportes,"$39,783,281.57","79,196"


,Categoría,Ticket Promedio ($)
0,Electrónica,$504.62
1,Ropa,$503.27
2,Hogar,$502.93
3,Alimentos,$502.39
4,Deportes,$502.33
